In [1]:
import os
import utm
import gdal
import time
import warnings
from osgeo import gdal
from osmapi import OsmApi
#http://python-overpy.readthedocs.io/en/latest/introduction.html
import overpy
api = overpy.Overpass()
# http://overpass-turbo.eu/# <- construct nice queries
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
# GDAL does not use python exceptions by default
gdal.UseExceptions()
#/*



In [2]:
#run this to get something we can render or use the visual product
#gdal_translate -ot Byte -scale 0 4096 0 255 /home/kat/code/hacking/osm/20170222_231253_0c27/20170222_231253_0c27_3B_AnalyticMS.tif test.tif

In [21]:
infile = "/home/katscott/code/hacking/PythonFromSpace/20170211_181115_0e16/20170211_181115_0e16_3B_Visual.tif"
#infile = "/home/kat/code/hacking/PythonFromSpace/test.tif"
gtif = gdal.Open(infile)
print gtif.GetProjectionRef()
arr = gtif.ReadAsArray()
trans = gtif.GetGeoTransform()
print trans
extent = (trans[0], trans[0] + gtif.RasterXSize*trans[1],
          trans[3] + gtif.RasterYSize*trans[5], trans[3])
pos = [5000,1900] # this is the pixel we want info about
plt.imshow(arr[:3,:,:].transpose((1, 2, 0)))#, extent=extent)
fig = plt.gcf()
plt.annotate("{0},{1}".format(pos[0],pos[1]), xy=pos, xycoords='data',
             xytext=(0.5, 0.5), textcoords='figure fraction',
             arrowprops=dict(arrowstyle="->"))
fig.set_size_inches(18.5, 10.5)
plt.show()

PROJCS["WGS 84 / UTM zone 10N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-123],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32610"]]
(540504.0, 3.0, 0.0, 4191378.0, 0.0, -3.0)


In [8]:
def pixel2utm(ds, x, y):
    """
    Returns utm coordinates from pixel x, y coords
    """
    xoff, a, b, yoff, d, e = ds.GetGeoTransform()
    xp = a * x + b * y + xoff
    yp = d * x + e * y + yoff
    return(xp, yp)

In [9]:
ds = gdal.Open(infile)
# take the GDAL info and make it into UTM
my_utm = pixel2utm(ds,pos[0],pos[1])
# convert UTM into Lat Long
# need to figure out how to get zone info
my_lla = utm.to_latlon(my_utm[0],my_utm[1],10,"N")
# do the lat long look up from OSM
geolocator = Nominatim()
# reverse look up the are based on lat lon
location = geolocator.reverse("{0},{1}".format(my_lla[0],my_lla[1]))
# print location info
print location.address
print location.raw
# get the OSM ID info
osm_id = int(location.raw["place_id"])
print osm_id
# create an interface to the OSM API
MyApi = OsmApi()
# Look up our position 
print MyApi.NodeGet(osm_id)


SF Winery, 200, California Avenue, SF, California, 94130, United States of America
{u'display_name': u'SF Winery, 200, California Avenue, SF, California, 94130, United States of America', u'place_id': u'69202063', u'lon': u'-122.36980175', u'boundingbox': [u'37.8168224', u'37.8179618', u'-122.3703775', u'-122.369226'], u'osm_type': u'way', u'licence': u'Data \xa9 OpenStreetMap contributors, ODbL 1.0. http://www.openstreetmap.org/copyright', u'osm_id': u'24274429', u'lat': u'37.8173921', u'address': {u'building': u'SF Winery', u'city': u'SF', u'house_number': u'200', u'country': u'United States of America', u'county': u'SF', u'state': u'California', u'postcode': u'94130', u'country_code': u'us', u'road': u'California Avenue'}}
69202063


/usr/local/lib/python2.7/dist-packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


{u'changeset': 3353549, u'uid': 147510, u'timestamp': datetime.datetime(2009, 12, 12, 3, 38, 31), u'lon': -84.562955, u'visible': True, u'version': 2, u'user': u'woodpeck_fixbot', u'lat': 33.592651, u'tag': {}, u'id': 69202063}


In [10]:
plt.imshow(arr[:3,:,:].transpose((1, 2, 0)))#, extent=extent)
fig = plt.gcf()
plt.annotate(location.address, xy=pos, xycoords='data',
             xytext=(0.25, 0.5), textcoords='figure fraction',color="red",
             arrowprops=dict(arrowstyle="->"))
fig.set_size_inches(18.5, 10.5)
plt.show()

In [11]:
#https://pcjericks.github.io/py-gdalogr-cookbook/
#This has been generated by the overpass-turbo wizard.
#The original search was:
#“natural=coastline”
#*/
#[out:json][timeout:25];
#// gather results
#(
#  way["natural"="coastline"]({{bbox}});
#);
#// print results
#out body;
#>;
#out skel qt;
# now let's see if we can get the coast line.
my_lla = utm.to_latlon(my_utm[0],my_utm[1],10,"N")
result = api.query("""
    [out:json][timeout:25];
    // gather results
    (
        way["natural"="coastline"]({{bbox}});
    );
    // print results
    out body;
    >;
    out skel qt;
    """)


ERROR: An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 4))



OverpassBadRequest: Error: line 5: parse error: Unknown query clause 
Error: line 5: parse error: ')' expected - '{' found. 

In [20]:
gtif.RasterXSize

8929